In [4]:
import pathlib
import pandas as pd


In [24]:
base_dir = pathlib.Path().cwd().parent
data_dir = base_dir / 'data' / 'raw'
processed_data_dir = base_dir / 'data' / 'processed'

CONTROL_USERS_FILENAME = "control_driver_tweets_uae_082019.jsonl"
IO_USERS_FILENAME = "uae_082019_tweets_csv_unhashed.csv"


In [25]:
df = pd.read_csv(data_dir / IO_USERS_FILENAME, sep=",")
df.head()


/tmp/ipykernel_2947207/1925020241.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_dir / IO_USERS_FILENAME, sep=",")


,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,1073994848330379264,1070186854861934592,mirna salah ⁦🇪🇬⁩,mirnasalah12,NaN,‏اهتمامك المزيف يبعث الكراهية بداخلى .. فأحذر ...,NaN,668,1178,2018-12-05,...,absent,absent,0.0,0.0,1.0,0.0,['يارب'],[],[],NaN
1,1097812240177205248,1077206810615037953,איילת ירון 🇮🇱,eilatyron,Israel,Fabulous ends in “us” coincidence? I think not. 😜,NaN,78,279,2018-12-24,...,absent,absent,0.0,0.0,1.0,0.0,[],[],[],NaN
2,1098392825715195905,1091348199980318721,Hoda Basem,HodaBasem90,United Arab Emirates,التدريب هو افضل المعلمين💪🏻💪🏻,NaN,14,43,2019-02-01,...,absent,absent,0.0,0.0,0.0,0.0,[],[],[],NaN
3,1074602743879340032,1069965231982559234,Ahmed Al-Jufairi🇶🇦,AhmedJufairi,NaN,لو كاانت لي دولة لجعلت ضحكه أمي نشيداً لها,NaN,142,112,2018-12-04,...,absent,absent,0.0,0.0,0.0,0.0,['الضرائب'],[],[],NaN
4,1100236248579538944,1061578239813083136,آدم محمد,AdamMoh2468,NaN,NaN,NaN,119,103,2018-11-11,...,absent,absent,0.0,0.0,0.0,0.0,[],['https://du3a.org'],[],NaN


In [26]:
io_drivers_df = df[['userid', 'tweetid', 'retweet_tweetid', 'retweet_userid', 'hashtags', 'urls', 'tweet_time', 'tweet_text']]
io_drivers_df.head()


,userid,tweetid,retweet_tweetid,retweet_userid,hashtags,urls,tweet_time,tweet_text
0,1070186854861934592,1073994848330379264,NaN,NaN,['يارب'],[],2018-12-15 17:35,وَتَوَلَّى عَنْهُمْ وَقَالَ يَا أَسَفَى عَلَى ...
1,1077206810615037953,1097812240177205248,NaN,NaN,[],[],2019-02-19 10:56,זה לא מה שאתם מסתכלים עליו שמשנה. זה מה שאתם ר...
2,1091348199980318721,1098392825715195905,NaN,NaN,[],[],2019-02-21 01:23,Lol😂 https://t.co/6mfLWN7aw5
3,1069965231982559234,1074602743879340032,NaN,NaN,['الضرائب'],[],2018-12-17 09:50,قانون الضرائب غير مبرر إطلاقا لماذا يتم فرض #ا...
4,1061578239813083136,1100236248579538944,NaN,NaN,[],['https://du3a.org'],2019-02-26 03:28,اللهم انا نسألك في هذا اليوم رحمة من عندك تهدى...


#### Filter control users

In [53]:
control_users_df = pd.read_json(data_dir / CONTROL_USERS_FILENAME, lines=True)


In [54]:
user_id_list = control_users_df.user.apply(lambda x: x['id']).values.tolist()
tweet_id_list = control_users_df.id.values.tolist()
def get_retweet_id(tuple_example):
    if not pd.isna(tuple_example):
        if 'id' in tuple_example:
            return tuple_example['id']
    return None
def get_user_retweet_id(tuple_example):
    if not pd.isna(tuple_example):
        if 'user' in tuple_example:
            return tuple_example['user']['id']
    return None
retweet_userid_list = control_users_df.retweeted_status.apply(lambda x: get_user_retweet_id(x)).values.tolist()
retweeted_id_list = control_users_df.retweeted_status.apply(lambda x: get_retweet_id(x)).values.tolist()
hashtags_list = control_users_df.entities.apply(lambda x: [elem['text'] for elem in x['hashtags']])
urls_list = control_users_df.entities.apply(lambda x: [elem['url'] for elem in x['urls']])
time_list = control_users_df.created_at.values.tolist()
text_list = control_users_df.full_text.values.tolist()


In [55]:
control_users_df = pd.DataFrame(data={'userid':user_id_list, 'tweetid': tweet_id_list, 
                                      'retweet_tweetid': retweeted_id_list, 'retweet_userid': retweet_userid_list,
                                      'hashtags': hashtags_list, 'urls': urls_list, 'tweet_time': time_list,
                                      'tweet_text': text_list})
def clean_RT_text(text_string, clean_flag):
    if clean_flag:
        tmp = text_string[2:].strip().split(':')[1:]  # delete "RT"string plus mention
        return ''.join(tmp)
    return text_string
control_users_df['tweet_text'] = control_users_df[['tweet_text', 'retweet_tweetid']].apply(lambda x: clean_RT_text(x['tweet_text'], not pd.isna(x['retweet_tweetid'])), axis=1)

control_users_df.head()


,userid,tweetid,retweet_tweetid,retweet_userid,hashtags,urls,tweet_time,tweet_text
0,268401843,117387437608673280,117329015202783232,370512955,"[ElBaradei, Egypt, Ibyar]",[],1316822320000000000,"RT @BaradeiCampaign: ""بنرددها جيل ورا جيل البر..."
1,268401843,117386921738641408,117386752557199360,248639940,[],[],1316822197000000000,RT @mooor_: انا مقتنعة -معرفش ليه-ان كل اللي ح...
2,268401843,117386145578491904,117386057036734464,251653606,[],[http://t.co/ISFDoJMK],1316822012000000000,RT @NoriAhmed: مسلحون يهاجمون أحد مقاهي «البور...
3,268401843,117384105364172800,117383289970499584,80274550,[EgyPresident],[http://t.co/7FmSmmZ2],1316821526000000000,RT @msheshtawy: لماذا أرفض حازم صلا أبو إسماعي...
4,268401843,117384065665081344,117383260652314624,251653606,[],[],1316821516000000000,RT @NoriAhmed: انا ماكونتش عايزة اعمل شير للصو...


In [91]:
control_users_df.to_csv(processed_data_dir / 'control_users.csv')
io_drivers_df.to_csv(processed_data_dir / 'io_drivers_users.csv')
